In [ ]:
# -------------------------------
# Imports
# -------------------------------
import os
from dotenv import load_dotenv
import pandas as pd
import urllib.parse
import base64
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display

# -------------------------------
# Password setup
# -------------------------------
load_dotenv()
PASSWORD = os.getenv("MY", "test")  # fallback password

# -------------------------------
# LOGIN UI
# -------------------------------
password_input = widgets.Password(
    description="Geslo:",
    placeholder="Vnesi Geslo",
    layout=widgets.Layout(width="300px")
)
login_button = widgets.Button(
    description="Odkleni",
    button_style="success",
    layout=widgets.Layout(width="125px")
)
login_output = widgets.Output()

login_box = widgets.VBox(
    [password_input, login_button, login_output],
    layout=widgets.Layout(
        align_items="center",
        justify_content="center",
        margin="100px"
    )
)
display(login_box)

# -------------------------------
# APP UI (hidden initially)
# -------------------------------
header_html = widgets.HTML("""
<div style="clear: both; margin-bottom: 20px;">
    <div>
        <h2><b>EXPORT</b>4PHOBS</h2>
        <img src="https://www.adria-ankaran.si//app/uploads/2025/10/logo-Adria.jpg" width="180" alt="">
    </div>
</div>
""")

# ✅ Voilà-safe iframe (wrapped in HTML widget)
iframe_widget = widgets.HTML("""
<iframe
    src="https://docs.google.com/spreadsheets/d/15HJ7wxyUmo-gcl5_y1M9gl4Ti-JSsYEJZCjoI76s-Xk/edit?gid=1385640257"
    width="100%" height="650" style="border:none;">
</iframe>
""")

refresh_label = widgets.Label("Last refreshed at: --:--")
main_out = widgets.Output()
spinner_out = widgets.Output()

reload_button = widgets.Button(
    description="Reload Content 🔄",
    button_style='warning',
    tooltip="Click to refresh all content",
    layout=widgets.Layout(width='200px')
)

app_ui = widgets.VBox([
    header_html,
    iframe_widget,
    widgets.HBox([reload_button], layout=widgets.Layout(margin='10px 0')),
    refresh_label,
    main_out
])
app_ui.layout.display = "none"  # hidden until login
display(app_ui)

# -------------------------------
# HELPER FUNCTIONS
# -------------------------------
def prepare_phobs_csv(df, hotel_id, los_code):
    if 'BAR' not in df.columns:
        df['BAR'] = 120
    df['BAR'] = df['BAR'].apply(lambda x: f"BAR{x}")
    df['Hotel_ID'] = hotel_id
    df['nicla'] = 0
    df['Yield'] = f"YIELD{los_code}"
    if 'Datum' not in df.columns:
        df['Datum'] = pd.Timestamp.today().strftime('%Y-%m-%d')
    return df[['Hotel_ID', 'Datum', 'nicla', 'BAR', 'Yield']]

def create_download_button(df, filename, description):
    download_out = widgets.Output()
    button = widgets.Button(
        description=description,
        button_style='info',
        layout=widgets.Layout(width='100%', height='40px', margin='2px 0')
    )

    def on_click(b):
        with download_out:
            download_out.clear_output(wait=True)
            csv_bytes = df.to_csv(index=False, header=False).encode()
            b64 = base64.b64encode(csv_bytes).decode()
            href = f"data:text/csv;base64,{b64}"
            html = f"""
            <div style='margin-top:6px;'>
                <a download="{filename}.csv" href="{href}" target="_blank"
                   style='text-decoration:none;color:white;
                          background-color:#5392ca;padding:6px 10px;
                          border-radius:4px;display:inline-block;'>
                📥  {filename}.csv
                </a>
            </div>
            """
            display(widgets.HTML(value=html))

    button.on_click(on_click)
    return widgets.VBox([button, download_out])

def initialize_phobs(*args):
    with main_out:
        main_out.clear_output()
        refresh_label.value = f"Last refreshed at: {datetime.now().strftime('%H:%M')}"
        display(widgets.HTML("<h3>PHOBS BAR Export .csv Generator</h3>"))

        spinner_html = """
        <div style="display:flex;align-items:center;gap:10px;margin:10px 0;">
          <div style="border:4px solid #f3f3f3;border-top:4px solid #3498db;
                      border-radius:50%;width:18px;height:18px;
                      animation:spin 1s linear infinite;"></div>
          <span>Loading hotel sheets...</span>
        </div>
        <style>
        @keyframes spin {0%{transform:rotate(0deg);}100%{transform:rotate(360deg);}}
        </style>
        """
        with spinner_out:
            spinner_out.clear_output()
            display(widgets.HTML(value=spinner_html))
        display(spinner_out)

        try:
            gsheet_id = "15HJ7wxyUmo-gcl5_y1M9gl4Ti-JSsYEJZCjoI76s-Xk"
            master_url = f"https://docs.google.com/spreadsheets/d/{gsheet_id}/gviz/tq?tqx=out:csv&sheet=PHOBS"
            master_df = pd.read_csv(master_url)
            hotels = [(r['Hotel_Name'], r['Hotel_ID'], r['YIELD_Code']) for _, r in master_df.iterrows()]
        except Exception as e:
            spinner_out.clear_output()
            display(widgets.HTML(value=f"<b style='color:red'>Failed to load master sheet:</b> {e}"))
            return

        buttons = []
        failed = []
        for hotel_name, hotel_id, los_code in hotels:
            try:
                sname = urllib.parse.quote(hotel_name)
                url = f"https://docs.google.com/spreadsheets/d/{gsheet_id}/gviz/tq?tqx=out:csv&sheet={sname}"
                df = pd.read_csv(url)
                df = prepare_phobs_csv(df, hotel_id, los_code)
                btn = create_download_button(df, f"{hotel_name}-Phobs", hotel_name.replace("_", " "))
                buttons.append(btn)
            except Exception as e:
                failed.append((hotel_name, str(e)))

        spinner_out.clear_output()
        if failed:
            display(widgets.HTML(value="<b>Some hotels failed to load:</b><ul>" +
                         "".join(f"<li>{h}: {e}</li>" for h, e in failed) + "</ul>"))

        grid = widgets.GridBox(
            children=buttons,
            layout=widgets.Layout(
                grid_template_columns="repeat(auto-fill, minmax(250px, 1fr))",
                grid_gap="8px",
                width="100%"
            )
        )
        display(grid)

reload_button.on_click(initialize_phobs)

# -------------------------------
# LOGIN LOGIC
# -------------------------------
def check_password(b):
    if password_input.value == PASSWORD:
        login_box.layout.display = "none"  # hide login
        app_ui.layout.display = ""         # show app
        initialize_phobs()                 # load data
    else:
        with login_output:
            login_output.clear_output()
            print("❌ Geslo ni pravilno. Poizkusi ponovno.")

login_button.on_click(check_password)
